<a href="https://colab.research.google.com/github/Linux-Server/AI_Engineering/blob/main/guidellm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install vllm
!pip install guidellm
!pip install bitsandbytes

In [3]:
!guidellm benchmark \
  --target "http://localhost:8000" \
  --rate-type sweep \
  --max-seconds 30 \
  --data "prompt_tokens=256,output_tokens=100"


Streaming output truncated to the last 5000 lines.
╭─ Benchmarks ─────────────────────────────────────────────────────────────────╮
│ [1… synch… (… Req:    1.4 req/s,    0.70s Lat,     1.0 Conc,      42 Comp, … │
│               Tok:  142.8 gen/s,  508.4 tot/s,  30.9ms TTFT,    6.7ms ITL, … │
│ [1… throu… (… Req:   17.5 req/s,   19.61s Lat,   344.1 Conc,     512 Comp, … │
│               Tok: 1754.3 gen/s, 6248.7 tot/s, 8240.5ms TTFT,  114.9ms ITL,… │
│ [1… const… (… Req:    3.4 req/s,    1.42s Lat,     4.8 Conc,     101 Comp, … │
│               Tok:  337.9 gen/s, 1203.9 tot/s,  37.9ms TTFT,   13.9ms ITL, … │
│ [1… const… (… Req:    5.3 req/s,    1.49s Lat,     8.0 Conc,     159 Comp, … │
│               Tok:  533.3 gen/s, 1899.9 tot/s,  38.3ms TTFT,   14.7ms ITL, … │
│ [1… const… (… Req:    7.3 req/s,    1.67s Lat,    12.2 Conc,     219 Comp, … │
│               Tok:  731.7 gen/s, 2605.7 tot/s,  45.7ms TTFT,   16.4ms ITL, … │
│ [1… const… (… Req:    9.3 req/s,    1.65s Lat,    15.3 C

In [5]:
from guidellm.benchmark import GenerativeBenchmarksReport

# Load results from file
report = GenerativeBenchmarksReport.load_file("benchmarks.json")

# Access individual benchmarks
for benchmark in report.benchmarks:
    # Print basic info
    print("--------------------------------------")
    print(f"Benchmark: {benchmark.id_}")
    print(f"Type: {benchmark.type_}")

    # Access metrics
    print(f"Avg RPS: {benchmark.metrics.requests_per_second.successful.mean}")
    print(f"p99 latency: {benchmark.metrics.request_latency.successful.percentiles.p99}")
    print(f"TTFT (p99): {benchmark.metrics.time_to_first_token_ms.successful.percentiles.p99}")
    print("--------------------------------------")

--------------------------------------
Benchmark: 7827dd1c-8654-4c59-853c-d177b5f5f8ff
Type: generative_benchmark
Avg RPS: 1.427974850534866
p99 latency: 0.7539141178131104
TTFT (p99): 86.23719215393066
--------------------------------------
--------------------------------------
Benchmark: f75784a3-beb6-434a-9417-c158dfc5fc84
Type: generative_benchmark
Avg RPS: 17.543381952412492
p99 latency: 28.851101636886597
TTFT (p99): 17123.49009513855
--------------------------------------
--------------------------------------
Benchmark: cfa6d635-09f8-4ac3-a149-6e164ea37e15
Type: generative_benchmark
Avg RPS: 3.379459499579738
p99 latency: 1.4757235050201416
TTFT (p99): 99.53093528747559
--------------------------------------
--------------------------------------
Benchmark: 38196397-8274-40e0-beb0-478867707cf9
Type: generative_benchmark
Avg RPS: 5.332891664756278
p99 latency: 1.5919578075408936
TTFT (p99): 119.20642852783203
--------------------------------------
------------------------------

In [7]:
import wandb
from guidellm.benchmark import GenerativeBenchmarksReport

wandb.init(project="llm-benchmarks", name="vllm-deployment-test")

report = GenerativeBenchmarksReport.load_file("benchmarks.json")

for i, benchmark in enumerate(report.benchmarks):
    wandb.log({
        "benchmark_id": benchmark.id_,
        "avg_rps": benchmark.metrics.requests_per_second.successful.mean,
        "p99_latency_s": benchmark.metrics.request_latency.successful.percentiles.p99,
        "ttft_p99_ms": benchmark.metrics.time_to_first_token_ms.successful.percentiles.p99,
        "iteration": i
    })

wandb.config.update({
    "model": "Llama-3-8B-Instruct",
    "deployment": "vLLM",
    "gpu_type": "A100 40GB",
    "batch_size": 8,
})
